## Individual run

In [17]:
import pickle
from tqdm.notebook import tqdm as tq


In [2]:
import sys
sys.path.append('../')
from util import corpus, tokenize, util, alignment, word2vec, retrieval

In [3]:
# df = util.decompress_pickle('../../data/word2vec/epo/corpus/epo_18_20_df_phrasal')
df = util.decompress_pickle('../../data/epo_corpus_all_years/pre_proc/epo_10_20_df_w_ncs')
df

,name,titles,en,de
0,EP03703859NWB1.xml,de\tMODULARER LEISTUNGSWANDLER MIT FLÜSSIGGEKÜ...,A modular power converter comprising : a therm...,"Modularer_Leistungswandler , der Folgendes umf..."
1,EP03703859NWB1.xml,de\tMODULARER LEISTUNGSWANDLER MIT FLÜSSIGGEKÜ...,"The modular_power_converter of claim NUM , whe...","Modularer_Leistungswandler nach Anspruch NUM ,..."
2,EP03703859NWB1.xml,de\tMODULARER LEISTUNGSWANDLER MIT FLÜSSIGGEKÜ...,"The modular_power_converter of claim NUM , whe...","Modularer_Leistungswandler nach Anspruch NUM ,..."
3,EP03703859NWB1.xml,de\tMODULARER LEISTUNGSWANDLER MIT FLÜSSIGGEKÜ...,"The modular_power_converter of claim NUM , whe...","Modularer_Leistungswandler nach Anspruch NUM ,..."
4,EP03703859NWB1.xml,de\tMODULARER LEISTUNGSWANDLER MIT FLÜSSIGGEKÜ...,"The modular_power_converter of claim NUM , whe...","Modularer_Leistungswandler nach Anspruch NUM ,..."
...,...,...,...,...
365019,EP15713208NWB1.xml,de\tVERFAHREN UND SYSTEME ZUM BESCHLEUNIGTEN H...,A motor vehicle NUM such that it comprises at ...,"Kraftfahrzeug , das mindestens einen Computer ..."
365020,EP15713208NWB1.xml,de\tVERFAHREN UND SYSTEME ZUM BESCHLEUNIGTEN H...,A system for the downloading of data comprisin...,"System zum Herunterladen von Daten , das ein H..."
365021,EP16181157NWB1.xml,de\tFAHRZEUG\ten\tVEHICLE\tfr\tVÉHICULE\t,A vehicle NUM comprising a power_reception_dev...,Fahrzeug NUM mit einem Leistungsempfangsgerät ...
365022,EP16181157NWB1.xml,de\tFAHRZEUG\ten\tVEHICLE\tfr\tVÉHICULE\t,The vehicle NUM according to any one of claims...,Fahrzeug NUM nach einem der Ansprüche NUM bis ...


In [4]:
df.en[0]

'A modular power converter comprising : a thermal_support NUM for a circuit , the support comprising a substrate having fluid_inlet NUM and outlet ports NUM and a circulation_path NUM coupled between the inlet and outlet ports , the inlet and outlet ports being configured to transmit a coolant for circulation through the circulation_path ; an interface_plate NUM configured to support a power_electronics_circuit NUM , the interface_plate being configured for mounting to the support adjacent to the coolant circulation_path for extraction of heat from the interface_plate during operation ; and a power_electronics_circuit configured for power_conversion supported on the interface_plate ; and a driver_circuit NUM for applying the drive_signals to the power_electronics_circuit , the driver_circuit being secured to and cooled by the thermal_support ; wherein the support at least partially defines an electrical_reference_plane , a mechanical_support , and a thermal_extraction_path for the powe

In [5]:
epo_corpus = corpus.Corpus(name='epo', 
                           doc=df,
                           tokenize_corpus=True,
                           load_vocab=True,
                           load_tokens=True)

10:44:33: Loading vocab
10:44:34: Loading tokens
10:44:42: Artefacts in the path: ../../data/word2vec/epo/
10:44:42: Sentences
	Lang	Size
	EN	365024
	DE	365024
10:44:42: Vocab
	Lang	Size
	EN	380934
	DE	552476


In [6]:
algn_dic = alignment.AlignmentDic(name='epo',
                                 corpus=epo_corpus,
                                 rerun=True)

10:44:42: Fetching alignment dictionary from path ../../data/word2vec/epo/


In [7]:
algn_dic.load_trans_dic()

10:44:43: Loading translation dictionary from path: ../../data/word2vec/epo/alg/transidx


In [8]:
len(algn_dic.idx2transidx.keys())

187415

In [9]:
len(algn_dic.idx2transidx), len(epo_corpus.idx2word)

(187415, 933410)

In [10]:
epo_corpus.idx2word[algn_dic.idx2transidx[epo_corpus.word2idx['vehicle']]]

'fahrzeug'

In [33]:
from imp import reload
reload(word2vec)

<module 'util.word2vec' from '../util/word2vec.py'>

In [7]:
w2v = word2vec.Word2Vec(name='epo', 
                        corpus=epo_corpus,
                        algn_dic=algn_dic,
                        rerun_data=True,
                        rerun_train=True,
                        remove_underscores=True
                       )


# w2v = word2vec.Word2Vec(name='epo', 
#                         corpus=epo_corpus,
#                         algn_dic=algn_dic,
#                         rerun_data=False,
#                         rerun_train=False,
#                        )

00:05:24: Fetching contents from path ../../data/word2vec/epo/


In [8]:
w2v.load_train_data()

00:05:31: Loading train data from path: ../../data/word2vec/epo/train/aligned_sentences


In [9]:
len(w2v.aligned_sentences)

3136108

In [10]:
from tqdm.notebook import tqdm as tq

wo_ncs_sent = []
for sent in tq(w2v.aligned_sentences):
    wo_ncs = []
    for token in sent:
        s = token.split('_')
        for _ in s:
            wo_ncs.append(_)
    wo_ncs_sent.append(wo_ncs)
# print(wo_ncs_sent)

In [11]:
len(wo_ncs_sent)

3136108

In [12]:
train_sent = wo_ncs_sent + w2v.aligned_sentences
len(train_sent)

6272216

In [12]:
from gensim.models import FastText

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [14]:
model = FastText(vector_size=300, 
                 window=5, 
                 min_count=1, 
                 sg=1)  # instantiate

00:07:07: FastText lifecycle event {'params': 'FastText(vocab=0, vector_size=300, alpha=0.025)', 'datetime': '2021-09-09T00:07:07.845494', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC 7.3.0]', 'platform': 'Linux-5.4.0-81-generic-x86_64-with-glibc2.10', 'event': 'created'}


In [ ]:
model.build_vocab(train_sent)

In [ ]:
model.train(corpus_iterable=train_sent, 
            total_examples=len(train_sent), 
            epochs=5)

In [17]:
model.save('epo_w_ncs_fasttext.model')

02:31:45: FastText lifecycle event {'fname_or_handle': 'epo_w_ncs_fasttext.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2021-09-09T02:31:45.424358', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC 7.3.0]', 'platform': 'Linux-5.4.0-81-generic-x86_64-with-glibc2.10', 'event': 'saving'}
02:31:45: not storing attribute vectors
02:31:45: storing np array 'vectors_vocab' to epo_w_ncs_fasttext.model.wv.vectors_vocab.npy
02:31:45: storing np array 'vectors_ngrams' to epo_w_ncs_fasttext.model.wv.vectors_ngrams.npy
02:31:47: not storing attribute buckets_word
02:31:47: storing np array 'syn1neg' to epo_w_ncs_fasttext.model.syn1neg.npy
02:31:47: not storing attribute cum_table
02:31:47: saved epo_w_ncs_fasttext.model


In [13]:
model = FastText.load('epo_w_ncs_fasttext.model')

10:45:40: loading FastText object from epo_w_ncs_fasttext.model
10:45:40: loading wv recursively from epo_w_ncs_fasttext.model.wv.* with mmap=None
10:45:40: loading vectors_vocab from epo_w_ncs_fasttext.model.wv.vectors_vocab.npy with mmap=None
10:45:42: loading vectors_ngrams from epo_w_ncs_fasttext.model.wv.vectors_ngrams.npy with mmap=None
10:45:46: setting ignored attribute vectors to None
10:45:46: setting ignored attribute buckets_word to None
10:47:29: loading syn1neg from epo_w_ncs_fasttext.model.syn1neg.npy with mmap=None
10:47:30: setting ignored attribute cum_table to None
10:47:36: FastText lifecycle event {'fname': 'epo_w_ncs_fasttext.model', 'datetime': '2021-09-09T10:47:36.904825', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC 7.3.0]', 'platform': 'Linux-5.4.0-84-generic-x86_64-with-glibc2.10', 'event': 'loaded'}


In [18]:
# Build vocab
wv_vocab = []
for idx in tq(range(len(model.wv))):
    wv_vocab.append(model.wv.index_to_key[idx])

In [19]:
len(wv_vocab)

745335

## Projection

In [19]:
from util import projection

In [20]:
projection.generate_tsv_files(w2v)


Vectors saved at: vectors_epo.tsv
Words saved at: words_epo.tsv
